# General math of surfaces

$S: (u, v) \to (x, y, z)$ 

$S(u, v) = [x(u, v), y(u, v), z(u, v)]$

$S_u = \frac{\partial S}{\partial u}$, $S_v = \frac{\partial S}{\partial v}$

Curve on surface:

$C: (t) \to (u,v)$ 

$C(t) = [u(t), v(t)]$

$T_C = S_u \frac{du}{dt} + S_v \frac{dv}{dt}$

Tangent plane:

$T_S(u,v) = S + \lambda S_u + \mu S_v$

Normal vector:

$N_S(u,v) = \frac{S_u \times S_v}{|S_u \times S_v|}$


# Pipes math
Directrix (spine): $A(t) = [x(t), y(t), z(t)]$

Generatrix (bevel): $C(s, t) = [x(s, t), y(s, t), 0]$, for uniform profile -- $C(s) = [x(s), y(s), 0]$

Generatrix frame: $Z(t) = T_A = A'(t)/|A'(t)|$ -- along spine tangent, $X(t), Y(t)$ -- orthogonal, e.g. $X = N_A, Y = B_A$ or $X = Z \times e_z,  Y = X \times Z$

Surface: $S(u, v) = A(u) + C(v, u)_x X(u) + C(v, u)_y Y(u)$

In [1]:
import numpy as np
import sympy as sp
from matplotlib.tri import Triangulation
from ipywidgets import widgets
import k3d
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

from myutils import *
from mycurves import *

In [2]:
x, y, z, t, s, u, v, r, a, b, c, d = sp.symbols("x, y, z, t, s, u, v, r, a, b, c, d")
P, P0, P1, P2, P3 = vec.symbols('P P0 P1 P2 P3')

In [3]:
def spine_frame(u, Q):
    # Verically-aligned spine frame: X axis parallel to world XY, Y axis approximately "up"
    Z = vec.normalize(sp.diff(Q, u))         # spine tangent direction
    X = vec.normalize(Z.cross(vector(0, 0, 1)))
    Y = X.cross(Z)
    return X, Y, Z

def Pipe(uv, Qa, Qc):
    u, v = uv
    X, Y, Z = spine_frame(u, Qa)
    return Qa + X * Qc[0] + Y * Qc[1]

def pipe_frame(uv, S):
    u, v = uv    
    U = vec.normalize(sp.diff(S, u))
    V = vec.normalize(sp.diff(S, v))
    N = U.cross(V)    
    return U, V, N

In [4]:
# export triangulate_pipe into myutils
def triangulate_pipe(nu, nv):
    for u in range(nu-1):
        for v in range(nv-1):
            idx = u * nv + v
            yield (idx, idx+nv, idx+nv+1)
            yield (idx, idx+nv+1, idx+1)
        idx = u * nv + nv - 1
        yield (idx, idx+nv, idx+1)
        yield (idx, idx+1, idx-nv+1)

In [1]:
# export eval_curve, eval_pipe into mycurves
def eval_curve(Q, u, segs=6):
    uu = np.linspace(0, 1, segs)
    points = eval(Q, {u: uu}, dtype=np.float32)
    return points

def eval_pipe(S, vrs, segs=(6, 6)):
    u, v = vrs
    nu = usegs+1
    nv = vsegs
    uu = np.linspace(0, 1, nu)
    vv = np.linspace(0, 1, nv)
    points = eval(S, {u: uu, v: vv}, dtype=np.float32).reshape(nu*nv, 3)
    tris = np.array(tuple(triangulate_pipe(nu, nv)), np.uint32)
    return points, tris, np.array(np.meshgrid(uu, vv)).T.reshape(-1,2)

## Plotting

In [58]:
def texture(coords, scale):
    checker = lambda uv: (int(uv[0]*scale[0]) + int(uv[1]*scale[1]))%2
    return map(checker, coords)

In [72]:
plot = k3d.plot()
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [73]:
Qa = vector(u*3, 0, sp.cos(u*2*sp.pi))
Qc = vector(0.2*sp.cos(v*2*sp.pi), 0.2*sp.sin(v*2*sp.pi), 0)
S = Pipe((u,v), Qa, Qc)

In [74]:
points = eval_curve(Qa, u, 36)
plot += k3d.line(points, name="spine")

In [76]:
points, tris, coords = eval_pipe(S, (u,v), 64, 32)
plot += k3d.mesh(points, tris, name="pipe", attribute=list(texture(coords, (32, 16))))

In [10]:
X, Y, Z = spine_frame(u, Qa)
origins = eval_curve(Qa, u, 4)
x_vectors = eval_curve(X, u, 4)
y_vectors = eval_curve(Y, u, 4)
z_vectors = eval_curve(Z, u, 4)
origins = np.concatenate([origins, origins, origins])
vectors = np.concatenate([x_vectors, y_vectors, z_vectors])
colors = [0xff0000] * 8 + [0x00ff00] * 8 + [0x0000ff] * 8
plot += k3d.vectors(origins, vectors, colors=colors, name='spine frame')   
    

In [11]:
U, V, N = pipe_frame((u,v), S)
origins, _ = eval_pipe(S, (u,v), 3, 4)
u_vectors, _ = eval_pipe(U, (u,v), 3, 4)
v_vectors, _ = eval_pipe(V, (u,v), 3, 4)
n_vectors, _ = eval_pipe(N, (u,v), 3, 4)
origins = np.concatenate([origins, origins, origins])
vectors = np.concatenate([u_vectors, v_vectors, n_vectors])
colors = [0xff0000] * 32 + [0x00ff00] * 32 + [0x0000ff] * 32
plot += k3d.vectors(origins, vectors, colors=colors, name='pipe frame')

Output()

# Reverse projection

Projecting points from fabric surface to pipe.

$P_{flat} \to S \to u_{proj}, v_{proj}, V_{displacement}$ 


### Projecting to Qa 

* make $Qa_{flat}$
* find closest point from $P_{flat}$ to $Qa_{flat}$ → $u_{proj}$
* find the distance from → $x_{dist}$

See [curves.ipynb](curves.ipynb)

## Projecting to Qc

* solve $Qc(v)_x = x_{dist}$ (for $Qc(v)_z > 0$) → $v_{proj}$
* final point = $P_{pipe} = S(u_{proj}, v_{proj})$ → $V_{displ}$ 


For circle/ellipse:

$v_{proj} = 1 - \frac{acos(x/r)}{2\pi}$
